In [7]:
#import libraries required for analysis
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps

In [10]:
#obtain AAF and EHF data from BQ, using measures.cpma_products as reference list

sql = """
SELECT
  rx.month,
  rx.pct,
  SUM(CASE
      WHEN cmpa.type = 'AAF' THEN items --calculate AAF items
      ELSE 0 END) AS aaf_items,
  SUM(CASE
      WHEN cmpa.type = 'AAF' THEN actual_cost -- calculate AAF cost
      ELSE 0 END) AS aaf_cost,
  SUM(CASE
      WHEN cmpa.type <> 'exclude' THEN items --calculate all CMPA items
      ELSE 0 END) AS all_items,
  SUM(CASE
      WHEN cmpa.type <> 'exclude' THEN actual_cost -- calculate all CMPA cost
      ELSE 0 END) AS all_cost,
  IEEE_DIVIDE(SUM(CASE
        WHEN cmpa.type = 'AAF' THEN items
        ELSE 0 END), SUM(CASE
      WHEN cmpa.type <> 'exclude' THEN items
      ELSE 0 END)) AS aaf_percent_items,   -- calculate AAF items proportion
  -- calculate AAF items proportion
  IEEE_DIVIDE(SUM(CASE
        WHEN cmpa.type = 'AAF' THEN actual_cost
        ELSE 0 END), SUM(CASE
      WHEN cmpa.type <> 'exclude' THEN actual_cost
      ELSE 0 END)) AS aaf_percent_cost  -- calculate AAF cost proportion
FROM
  hscic.normalised_prescribing_standard AS rx
JOIN
  measures.cmpa_products AS cmpa --join with CPMA products table
ON
  rx.bnf_code=cmpa.bnf_code
WHERE
  rx.bnf_code IN (
  SELECT
    bnf_code
  FROM
    measures.cmpa_products)
GROUP BY
  month,
  pct
ORDER BY
  month
"""
cmpa_df = bq.cached_read(sql, csv_path='rte.csv')
cmpa_df.head(5)

,month,pct,aaf_items,aaf_cost,all_items,all_cost,aaf_percent_items,aaf_percent_cost
0,2010-08-01,99K,25,3966.54,48,5111.59,0.520833,0.775989
1,2010-08-01,05X,0,0.00,12,554.93,0.000000,0.000000
2,2010-08-01,04M,24,2362.50,65,4601.03,0.369231,0.513472
3,2010-08-01,07R,61,10419.99,113,13271.97,0.539823,0.785113
4,2010-08-01,03N,62,11323.13,217,19691.90,0.285714,0.575015
